In [2]:
import torch
import sys
from swift.llm import get_model_tokenizer
dirs = ["/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm"]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)
from custom import CustomModelType, CustomTemplateType
model_type = CustomModelType.mixtral_moe_7b_instruct_gptq_int4
model, tokenizer = get_model_tokenizer(
    model_type, 
    load_model=False
)


2024-04-02 19:02:36,428 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-04-02 19:02:36,431 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-04-02 19:02:36,467 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 ad87f4a589251d86333765b92ab59e41 and a total number of 972 components indexed
[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mixtral",
  "num_attention_heads": 32,
  "num_experts_per_tok": 2,
  "num_hidde

In [3]:
import jsonlines
from tqdm.notebook import tqdm

dict_list = []
version = "2"
with jsonlines.open(f"data{version}.jsonl", mode="r") as file_jsonl:
    for item in file_jsonl.iter():
        dict_list.append(item)

lens1 = []
lens2 = []

for item in tqdm(dict_list):
    lens1.append(len(tokenizer(item["query"])["input_ids"]))
    # lens2.append(len(tokenizer.tokenize(item["query"])))



  0%|          | 0/12192 [00:00<?, ?it/s]

In [4]:
# 3705

# [3675, 3674, 3671, 3665, 3662, 3662, 3640, 3638, 3636, 3632, 3616, 3604]


sorted(lens1, reverse=True)[0: 20]


[4240,
 1484,
 1365,
 1359,
 1355,
 1342,
 1341,
 1332,
 1269,
 1269,
 1247,
 1243,
 1235,
 1224,
 1224,
 1222,
 1221,
 1221,
 1219,
 1214]

修改较长的句子

显卡 24GB

7B模型，长度控制在3600以下 


In [5]:
del_ids = []
for i, item in enumerate(lens1):
    if item == 1484:
        del_ids.append(i)

del_ids


[5073]

In [13]:
# print(len(tokenizer(dict_list[4157]["query"])["input_ids"]))
dict_list[1862]["query"]


"According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.\n\nCLAIM:\nPublication date: 2020-02-20\nContent: Social media posts claim Chloroquine, an antimalarial drug, can cure COVID-19.\n\nPRIOR KNOWLEDGE:\nInformation Expansion and Summary:\n\nInformation 1 describes the FDA's review of safety issues related to the use of hydroxychloroquine and chloroquine to treat hospitalized patients with COVID-19. The FDA has revoked the emergency use authorization (EUA) for these drugs in treating COVID-19 due to serious heart rhythm problems and other safety issues, including blood and lymph system disorders, kidney injuries, and liver problems and failure. The FDA also warns of increased use of these medicines through outpatient prescriptions and reports of serious heart rhythm problems in patients with COV

In [6]:
print(dict_list[5073]["query"])


According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.

CLAIM:
Publication date: 2020-07-06
Content: No one has died from the coronavirus; there are no antibodies that identify SARS-CoV-2; pathologists have no means of verifying whether a person was infected with SARS-CoV-2; SARS-CoV-2 has not fulfilled Koch’s postulates.

PRIOR KNOWLEDGE:
Information Expansion and Summary:

The claim that no one has died from the coronavirus, there are no antibodies that identify SARS-CoV-2, pathologists have no means of verifying whether a person was infected with SARS-CoV-2, and SARS-CoV-2 has not fulfilled Koch's postulates is false. Monoclonal antibodies that bind to SARS-CoV-2 have been discovered and reported in published studies, and pathologists have been using such antibodies, as well as other techniques